In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/sf-crime/train.csv.zip')
test_df = pd.read_csv('/kaggle/input/sf-crime/test.csv.zip')

In [ ]:
def scheduler(epoch, lr):
...   if epoch < 10:
...     return lr*10
...   else:
...     return lr * tf.math.exp(-0.1)

In [ ]:
train_df.head()

In [ ]:
train_df.Category.nunique()

In [ ]:
test_df.head()

In [ ]:
train_data = train_df.drop(['Descript','Resolution','Address'],axis=1)
test_data = test_df.drop(['Id','Address'],axis=1)

In [ ]:
train_data.head()

In [ ]:
x = train_data.Dates
year = []
month = []
day = []
time = []
for i in range(len(x)):
    year.append(int(x[i][:4]))
    month.append(int(x[i][5:7]))
    day.append(int(x[i][8:10]))
    time.append(int(x[i][11:13])*100+int(x[i][14:16]))
train_data['year'] = year
train_data['month'] = month
train_data['day'] = day
train_data['time'] = time

In [ ]:
x = test_data.Dates
year = []
month = []
day = []
time = []
for i in range(len(x)):
    year.append(int(x[i][:4]))
    month.append(int(x[i][5:7]))
    day.append(int(x[i][8:10]))
    time.append(int(x[i][11:13])*100+int(x[i][14:16]))
test_data['year'] = year
test_data['month'] = month
test_data['day'] = day
test_data['time'] = time

In [ ]:
train_data.head()

In [ ]:
train_data.drop('Dates',axis=1,inplace=True)
test_data.drop('Dates',axis=1,inplace=True)

In [ ]:
dm1 = pd.get_dummies(train_data.DayOfWeek)
train_data[dm1.columns] = dm1
dm2 = pd.get_dummies(test_data.DayOfWeek)
test_data[dm2.columns] = dm2
dm1 = pd.get_dummies(train_data.PdDistrict)
train_data[dm1.columns] = dm1
dm2 = pd.get_dummies(test_data.PdDistrict)
test_data[dm2.columns] = dm2

In [ ]:
train_data.drop('DayOfWeek',axis=1,inplace=True)
test_data.drop('DayOfWeek',axis=1,inplace=True)
train_data.drop('PdDistrict',axis=1,inplace=True)
test_data.drop('PdDistrict',axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
X = train_data.drop('Category',axis=1)
y = pd.get_dummies(train_data.Category)
Xtest = test_data

In [ ]:
y

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
X = sc.fit_transform(X)
Xtest = sc.fit_transform(Xtest)

print(X.shape,Xtest.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1)

In [ ]:
  from tensorflow.keras.layers import Dense,Dropout,Activation
  from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(23,),bias_regularizer=tf.keras.regularizers.l1_l2(l1=0.01,l2=0.01)))
model.add(Activation('relu'))
model.add(Dropout(0.15))
model.add(Dense(64,bias_regularizer=tf.keras.regularizers.l1_l2(l1=0.01,l2=0.01)))
model.add(Activation('tanh'))
model.add(Dropout(0.2))
model.add(Dense(39))
model.add(Activation('softmax'))

In [ ]:
c1 = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    min_delta=0,
    patience=10,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)
c2 = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,batch_size=1024,epochs=1000,verbose=2,validation_data=(X_test,y_test),callbacks=[c1])

In [ ]:

col_names=y.columns

In [ ]:
sample.head()

In [ ]:
pred=model.predict(Xtest)

In [ ]:
m = np.max(pred, axis=1).reshape(-1, 1)
predicted = np.array((pred == m), dtype='int32')
predicted

In [ ]:
submission = pd.DataFrame()
submission['Id']=test_df['Id']
for i, entry in enumerate(col_names):
    submission[entry] = predicted[:,i]

submission.set_index('Id',inplace=True)    

In [ ]:
submission.head()

In [ ]:
submission.to_csv('Submission.csv')